In [5]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
import pymupdf4llm
import gradio as gr

In [6]:
load_dotenv(override=True)

True

In [47]:
deepseek_client = OpenAI(
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)
model_name = "deepseek/deepseek-chat-v3.1:free"

I am using Gotify here for the push notification testing on my local network with the gotify app installed on my tab and connected with the my local network url [https://gotify.net](https://gotify.net/)

In [8]:
gotify_token = os.getenv("GOTIFY_TOKEN")
gotify_url = f"http://192.168.0.110:80/message?token={gotify_token}"

In [9]:
def push(message):
    requests.post(gotify_url, json={"message": message, "priority": 5, "title": "PDF Chatbot"})

In [10]:
push("PDF Chatbot started")

In [40]:
def record_user_details(user_email, user_name="Unknown", user_notes="Unknown"):
    push(f"Recording interest from {user_name} with email: {user_email} and notes: {user_notes}")
    return {"recorded": "ok"}

In [13]:
def record_unknown_question(question):
    push(f"Recording unknown question: {question} that I couldn't answer")
    return {"recorded": "ok"}

In [36]:
record_unknown_question("What is the meaning of life?")

{'recorded': 'ok'}

In [41]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "user_email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "user_name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "user_notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["user_email"],
        "additionalProperties": False
    }
}

In [16]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [42]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [43]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'user_email': {'type': 'string',
      'description': 'The email address of this user'},
     'user_name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'user_notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['user_email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
   

In [44]:
def handle_tool_calls(tool_calls):
    results = []
    available_functions = {
        "record_user_details": record_user_details,
        "record_unknown_question": record_unknown_question
    }
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        print(f"Calling function {function_name} with args {function_args}", flush=True)

        function_response = function_to_call(**function_args)
        results.append({"tool_call_id": tool_call.id, "role": "tool", "content": json.dumps(function_response)})
    
    return results

In [21]:
resume_md = pymupdf4llm.to_markdown("./me/Shubham_Lad_Software_Engineer_II.pdf")

with open("./me/summary.txt", "r") as f:
    summary = f.read()
    f.close()

name = "Shubham Lad"

In [23]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and Resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{resume_md}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [48]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False

    while not done:
        print(messages[1:], flush=True)

        # We are calling the LLM with tool description passed to it
        response = deepseek_client.chat.completions.create(
            model=model_name,
            messages=messages,
            tools=tools
        )
        finish_reason = response.choices[0].finish_reason

        # If the LLM returns a tool call, we need to handle it
        if finish_reason == "tool_calls":
            response_message = response.choices[0].message
            tool_calls = response_message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(response_message)
            messages.extend(results)
        else:
            done = True

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'hi there'}]
[{'role': 'user', 'metadata': None, 'content': 'hi there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hi! 👋 Thanks for stopping by my website! I'm Shubham Lad, a Software Engineer with experience across product and service-based companies. I specialize in backend development, AWS infrastructure, and building scalable systems.\n\nWhat brings you here today? Are you interested in discussing potential opportunities, learning more about my technical expertise, or do you have something specific you'd like to ask about my background?", 'options': None}, {'role': 'user', 'content': 'list down your expertise'}]
[{'role': 'user', 'metadata': None, 'content': 'hi there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hi! 👋 Thanks for stopping by my website! I'm Shubham Lad, a Software Engineer with experience across product and service-based companies. I specialize in backend development, AWS infrastructu

## Command to Deploy

```bash
gradio deploy
```

- You must be logged in to Hugging Face with your token beforehand (`huggingface-cli login`).
- The deploy command will prompt for your Hugging Face credentials or use your token if already set.
- It packages and sends your local app to Hugging Face Spaces, making deployment simpler and faster.

This minimizes the deployment steps to a single command, provided authentication is handled and your app files are present locally.